<a href="https://colab.research.google.com/github/myleskeller/ColabOffloading/blob/main/Pix2Vox_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone the Code Repository

In [9]:
%cd /content/
!git clone https://github.com/hzxie/Pix2Vox.git

/content
fatal: destination path 'Pix2Vox' already exists and is not an empty directory.


# Install Python Denpendencies

In [10]:
%cd /content/Pix2Vox

# removes problematic requirements
with open("/content/Pix2Vox/requirements.txt", "r") as f:
    lines = f.readlines()
with open("/content/Pix2Vox/requirements.txt", "w") as f:
    for line in lines:
        if not "pprint" in line:
          if not "argparse" in line:
            f.write(line)

!pip install -r requirements.txt

/content/Pix2Vox


# Update Settings in config.py

In [11]:
old_prefix = "/home/hzxie/Datasets/"
new_prefix = "./datasets/"

with open("config.py", "r") as f:
    lines = f.readlines()
with open("config.py", "w") as f:
    for line in lines:
      if old_prefix in line:
        # changes path prefix "/home/hzxie/Datasets/..." to "./datasets/..."
        line = line.replace(old_prefix, new_prefix)
      f.write(line)

print("Changed path prefix "+old_prefix+"... to "+new_prefix+"...")

changed path prefix /home/hzxie/Datasets/... to ./datasets/...


# Datasets

In [12]:
#@title Select Datasets
pascal_3d = False #@param {type:"boolean"}
pix_3d = True #@param {type:"boolean"}
shape_net = False #@param {type:"boolean"}

## Download

In [13]:
if shape_net:
  # ShapeNet rendering images: 
  !wget http://cvgl.stanford.edu/data2/ShapeNetRendering.tgz -P /content/Pix2Vox/datasets/ShapeNet
  # ShapeNet voxelized models: 
  !wget http://cvgl.stanford.edu/data2/ShapeNetVox32.tgz -P /content/Pix2Vox/datasets/ShapeNet

if pix_3d:
  # Pix3D images & voxelized models: 
  !wget http://pix3d.csail.mit.edu/data/pix3d.zip -P /content/Pix2Vox/datasets/Pix3D

if pascal_3d:
  # PASCAL3D images & voxelized models: 
  !wget ftp://cs.stanford.edu/cs/cvgl/PASCAL3D+_release1.0.zip -O /content/Pix2Vox/datasets/PASCAL3D.zip

--2021-10-19 20:43:54--  http://pix3d.csail.mit.edu/data/pix3d.zip
Resolving pix3d.csail.mit.edu (pix3d.csail.mit.edu)... 128.52.128.175
Connecting to pix3d.csail.mit.edu (pix3d.csail.mit.edu)|128.52.128.175|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3783261880 (3.5G) [application/zip]
Saving to: ‘/content/Pix2Vox/datasets/Pix3D/pix3d.zip’

pix3d.zip           100%[===================>]   3.52G  38.4MB/s    in 94s     

2021-10-19 20:45:29 (38.4 MB/s) - ‘/content/Pix2Vox/datasets/Pix3D/pix3d.zip’ saved [3783261880/3783261880]



## Decompress

In [14]:
if shape_net:
  # extract ShapeNet images & voxels from .tgz
  !tar -zxvf /content/Pix2Vox/datasets/ShapeNet/ShapeNetVox32.tgz --directory /content/Pix2Vox/datasets/ShapeNet
  !tar -zxvf /content/Pix2Vox/datasets/ShapeNet/ShapeNetRendering.tgz --directory /content/Pix2Vox/datasets/ShapeNet

if pix_3d:
  # extract Pix3D images & voxels from .zip
  !unzip /content/Pix2Vox/datasets/Pix3D/pix3d.zip -d /content/Pix2Vox/datasets/Pix3D

if pascal_3d:
  # extract PASCAL3D images & voxels from .zip
  !unzip /content/Pix2Vox/datasets/PASCAL3D.zip -d /content/Pix2Vox/datasets/
  !mv /content/Pix2Vox/datasets/PASCAL3D+_release1.0 /content/Pix2Vox/datasets/PASCAL3D

Streaming output truncated to the last 5000 lines.
  inflating: /content/Pix2Vox/datasets/Pix3D/mask/sofa/1654.png  
  inflating: /content/Pix2Vox/datasets/Pix3D/mask/sofa/1592.png  
  inflating: /content/Pix2Vox/datasets/Pix3D/mask/sofa/1235.png  
  inflating: /content/Pix2Vox/datasets/Pix3D/mask/sofa/0807.png  
  inflating: /content/Pix2Vox/datasets/Pix3D/mask/sofa/0475.png  
  inflating: /content/Pix2Vox/datasets/Pix3D/mask/sofa/0303.png  
  inflating: /content/Pix2Vox/datasets/Pix3D/mask/sofa/1931.png  
  inflating: /content/Pix2Vox/datasets/Pix3D/mask/sofa/1543.png  
  inflating: /content/Pix2Vox/datasets/Pix3D/mask/sofa/1122.png  
  inflating: /content/Pix2Vox/datasets/Pix3D/mask/sofa/1685.png  
  inflating: /content/Pix2Vox/datasets/Pix3D/mask/sofa/0762.png  
  inflating: /content/Pix2Vox/datasets/Pix3D/mask/sofa/0989.png  
  inflating: /content/Pix2Vox/datasets/Pix3D/mask/sofa/0958.png  
  inflating: /content/Pix2Vox/datasets/Pix3D/mask/sofa/1653.png  
  inflating: /content/Pix

## Generate .binvox files for Pix3d

In [31]:
# download binvox
# !wget https://www.patrickmin.com/binvox/download.php?id=4 -O /content/binvox
# !wget --trust-server-names https://www.patrickmin.com/binvox/download.php?id=4
# !curl -O -J -L https://www.patrickmin.com/binvox/download.php?id=4
!wget -O binvox https://www.patrickmin.com/binvox/download.php?id=4

# run binvox
%cd /content/
!chmod +x ./binvox
!./binvox

--2021-10-19 21:07:14--  https://www.patrickmin.com/binvox/download.php?id=4
Resolving www.patrickmin.com (www.patrickmin.com)... 139.162.224.231
Connecting to www.patrickmin.com (www.patrickmin.com)|139.162.224.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 481 [text/html]
Saving to: ‘binvox’

binvox              100%[===================>]     481  --.-KB/s    in 0s      

2021-10-19 21:07:14 (46.0 MB/s) - ‘binvox’ saved [481/481]

/content
./binvox: line 2: syntax error near unexpected token `<'
./binvox: line 2: `<html><head>'


# Dowload Pretrained Models

In [15]:
# directly downloads model from Google Drive
!pip install gdown
import gdown

URL='https://drive.google.com/u/0/uc?id=1DlVD6vXT7htOGsTlYonigblonP5Xp5VX&export=download'
output = 'Pix2Vox-A-ShapeNet.pth'
gdown.download(URL, output, quiet=False) 

Downloading...
From: https://drive.google.com/u/0/uc?id=1DlVD6vXT7htOGsTlYonigblonP5Xp5VX&export=download
To: /content/Pix2Vox/Pix2Vox-A-ShapeNet.pth
100%|██████████| 457M/457M [00:06<00:00, 74.1MB/s]


'Pix2Vox-A-ShapeNet.pth'

# Testing pix2vox

In [ ]:
!python3 runner.py --test --weights=/content/Pix2Vox/Pix2Vox-A-ShapeNet.pth